Phase 1: Classification

we want to train a classifier that can predict if a sub-cube of the hydrodynamic box is empty or not.

In [3]:
cd "F:\data"


F:\data


In [4]:
##reading the galaxyy data(target)


import h5py
import numpy as np
import pandas as pd
f = h5py.File('groups_135.Subhalo.SubhaloPos.hdf5' , 'r') #data includes galaxy position in simulation box for 135th snapshot
head_subhalo = f['Subhalo']
data_subhalo = np.array(head_subhalo['SubhaloPos'])
print(data_subhalo.shape)
position = pd.DataFrame(data_subhalo)
position.columns = ['x' , 'y' , 'z']
position.head(10)

(121209, 3)


,x,y,z
0,900.425293,26286.689453,18321.949219
1,181.120392,24638.792969,16896.896484
2,826.923584,26747.314453,17363.390625
3,1492.193726,26514.935547,19381.691406
4,839.730957,26543.013672,15552.993164
5,267.409637,26475.843750,15855.078125
6,963.795166,26495.972656,17794.994141
7,1171.580200,26447.220703,18336.576172
8,74993.562500,24189.980469,16774.009766
9,1632.506470,25474.320312,18378.683594


In [5]:
##making sub-cubes

def process(position):
    position['x_b'] = np.floor(position['x']/(75000/100))
    position['y_b'] = np.floor(position['y']/(75000/100))
    position['z_b'] = np.floor(position['z']/(75000/100))
    position['x_b'] = position['x_b'].astype(int)
    position['y_b'] = position['y_b'].astype(int)
    position['z_b'] = position['z_b'].astype(int)
    
    
    return position
    
position = process(position)
position['c'] = 1
position.head(10)

,x,y,z,x_b,y_b,z_b,c
0,900.425293,26286.689453,18321.949219,1,35,24,1
1,181.120392,24638.792969,16896.896484,0,32,22,1
2,826.923584,26747.314453,17363.390625,1,35,23,1
3,1492.193726,26514.935547,19381.691406,1,35,25,1
4,839.730957,26543.013672,15552.993164,1,35,20,1
5,267.409637,26475.843750,15855.078125,0,35,21,1
6,963.795166,26495.972656,17794.994141,1,35,23,1
7,1171.580200,26447.220703,18336.576172,1,35,24,1
8,74993.562500,24189.980469,16774.009766,99,32,22,1
9,1632.506470,25474.320312,18378.683594,2,33,24,1


In [6]:
counts = position.groupby(['x_b' , 'y_b' , 'z_b'])['c'].count().reset_index(name = "count")
counts.head(180)
print(position.shape)
print(counts.shape)

(121209, 7)
(73908, 4)


In [7]:
##making an array that shows the number of the galaxies in each sub-cube

def dataframe_to_array(data , shape):
    ids = np.ravel_multi_index(data[['x_b' , 'y_b' , 'z_b']].values.T , shape)
    val = data['count'].values
    return np.bincount(ids , val , minlength = np.prod(shape)).reshape(shape)
arr_subhalo = dataframe_to_array(counts , (100,100,100))

print(arr_subhalo.shape)
print(arr_subhalo)


(100, 100, 100)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 

In [8]:
##making a binary array that shows if a sub-cube is empty or not


c=0
Bin_arr= np.zeros((100,100,100))
for i in range(100):
    for j in range(100):
        for k in range(100):
            if arr_subhalo[i,j,k] >0 :
                Bin_arr [i,j,k] = 1
                c+=1
            if arr_subhalo[i,j,k] ==0 :
                Bin_arr [i,j,k] = 0
            
        
print(c)
Bin_arr.shape

73908


(100, 100, 100)

In [9]:
##reduce the size of the box(temp)

sBin_arr = Bin_arr[0:30, 0:30, 0:30]

print(sBin_arr.shape)
print(sBin_arr)

(30, 30, 30)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 1. 1.]
  [0. 1. 0. ... 0. 1. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 1. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 ...

 [[0. 1. 0. ... 1. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 1. 0.]
  [1. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]

In [10]:
cd "F:\data"


F:\data


In [11]:
##reading the dark matter data(something like arr_subhalo)

import h5py
import numpy as np
import pandas as pd
f = h5py.File('data_100.100.h5' , 'r') #data includes galaxy position in simulation box for 135th snapshot
dm_arr = f['dm_arr']
print(dm_arr.shape)
print(dm_arr[0][0][2])


(100, 100, 100)
7.0


In [12]:
##reduce the size of the box(temp)

sdm_arr = dm_arr[0:30, 0:30, 0:30]

print(sdm_arr.shape)
print(sdm_arr)

(30, 30, 30)
[[[4.000e+00 5.000e+00 7.000e+00 ... 3.000e+00 6.000e+00 1.000e+01]
  [7.000e+00 7.000e+00 1.100e+01 ... 5.000e+00 6.000e+00 9.000e+00]
  [4.000e+00 4.000e+00 7.000e+00 ... 6.000e+00 1.400e+01 7.000e+00]
  ...
  [3.200e+01 2.800e+01 7.000e+00 ... 8.600e+01 9.700e+01 5.330e+02]
  [2.530e+02 3.900e+01 1.100e+01 ... 1.600e+01 3.830e+02 1.163e+03]
  [7.800e+01 5.850e+02 1.900e+01 ... 1.570e+02 7.890e+02 4.574e+03]]

 [[6.000e+00 6.000e+00 7.000e+00 ... 2.600e+01 2.400e+01 2.600e+01]
  [9.000e+00 4.000e+00 5.000e+00 ... 6.000e+00 4.000e+00 7.000e+00]
  [7.000e+00 1.100e+01 7.000e+00 ... 8.000e+00 5.000e+00 6.000e+00]
  ...
  [2.500e+01 4.300e+01 8.100e+01 ... 8.200e+01 5.200e+01 3.484e+03]
  [2.400e+01 1.000e+01 1.300e+01 ... 2.400e+01 2.160e+02 1.756e+03]
  [2.300e+01 1.500e+01 6.700e+01 ... 6.200e+01 8.140e+02 1.462e+03]]

 [[6.000e+00 3.000e+00 5.000e+00 ... 2.800e+01 2.200e+01 3.700e+01]
  [4.000e+00 5.000e+00 1.000e+01 ... 3.000e+01 4.300e+01 3.300e+01]
  [6.000e+00 5.000e

In [13]:
##making the feature matrix and the target matrix:  X:the N-body simulation data, Y:the hydrodynamic simulation data

X = np.zeros((23*23*23,8,8,8))
Y = np.zeros((23*23*23,1))

l = 0
for i in range(4,27):
    for j in range(4,27):
        for k in range(4,27):
            X [l,0:8,0:8,0:8] = sdm_arr[i-4:i +4 , j-4: j +4 , k-4:k+4]
            Y [l,0] = sBin_arr[i , j , k]
            l +=1
                
#print(X)

In [14]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(X,Y, random_state = 0)

In [15]:
print(Bin_arr.shape)
print(dm_arr.shape)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(100, 100, 100)
(100, 100, 100)
(9125, 8, 8, 8)
(9125, 1)
(3042, 8, 8, 8)
(3042, 1)


In [16]:
##train a CNN with activation function=sigmoid, optimizer=adam, loss=sparse_categorical_crossentropy, batch size=100 and 10 epochs.

import tensorflow as tf
from tensorflow import keras

new_model = keras.models.Sequential()

new_model.add(keras.layers.Conv3D(32, (3,3,1)  , activation = 'relu' , padding = 'same' , input_shape=(8,8,8,1)))
new_model.add(keras.layers.Conv3D(32, (3,3,1) , padding = 'same' , activation = 'relu' ))
new_model.add(keras.layers.MaxPooling3D((2,2,1) , padding = 'same'))
new_model.add(keras.layers.Dropout(0.25))
new_model.add(keras.layers.Conv3D(64, (3,3,1) , padding = 'same' , activation = 'relu' ))
new_model.add(keras.layers.Conv3D(64, (3,3,1) , padding = 'same' , activation = 'relu' ))
new_model.add(keras.layers.MaxPooling3D((2,2,1) , padding = 'same'))
new_model.add(keras.layers.Dropout(0.25))
new_model.add(keras.layers.Flatten())

new_model.add(keras.layers.Dense(120 , activation = 'relu'))
new_model.add(keras.layers.Dense(60 , activation = 'relu'))
new_model.add(keras.layers.Dense(2, activation = 'sigmoid'))
              
new_model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy' ])

new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 8, 8, 8, 32)       320       
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 8, 8, 8, 32)       9248      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 4, 8, 32)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 8, 32)       0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 4, 4, 8, 64)       18496     
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 4, 4, 8, 64)       36928     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 2, 2, 8, 64)       0         
__________

In [17]:
X_train = X_train.reshape(-1,8,8,8,1)
X_test = X_test.reshape(-1,8,8,8,1)

In [18]:
history = new_model.fit(X_train , Y_train, epochs = 10 , batch_size = 100, validation_data = (X_test,Y_test))

Train on 9125 samples, validate on 3042 samples
Epoch 1/10
9125/9125 [==============================] - 71s 8ms/sample - loss: 1.5995 - acc: 0.8990 - val_loss: 1.4942 - val_acc: 0.9073
Epoch 2/10
9125/9125 [==============================] - 72s 8ms/sample - loss: 1.6021 - acc: 0.9006 - val_loss: 1.4942 - val_acc: 0.9073
Epoch 3/10
9125/9125 [==============================] - 69s 8ms/sample - loss: 1.6021 - acc: 0.9006 - val_loss: 1.4942 - val_acc: 0.9073
Epoch 4/10
9125/9125 [==============================] - 65s 7ms/sample - loss: 1.6021 - acc: 0.9006 - val_loss: 1.4942 - val_acc: 0.9073
Epoch 5/10
9125/9125 [==============================] - 64s 7ms/sample - loss: 1.6021 - acc: 0.9006 - val_loss: 1.4942 - val_acc: 0.9073
Epoch 6/10
9125/9125 [==============================] - 66s 7ms/sample - loss: 1.6021 - acc: 0.9006 - val_loss: 1.4942 - val_acc: 0.9073
Epoch 7/10
9125/9125 [==============================] - 61s 7ms/sample - loss: 1.6021 - acc: 0.9006 - val_loss: 1.4942 - val_acc: 

In [20]:
from sklearn.metrics import confusion_matrix

Y_predicted = np.argmax( new_model.predict(X_test ), axis=1 )
cm = confusion_matrix(Y_test, Y_predicted)
print(cm)

[[2760    0]
 [ 282    0]]


In [18]:
from sklearn.metrics import classification_report
Y_temp=Y[:len(Y_predicted)]
clf_report = classification_report(Y_temp, Y_predicted)
print(clf_report)

              precision    recall  f1-score   support

         0.0       0.87      0.03      0.06      2794
         1.0       0.08      0.95      0.15       248

    accuracy                           0.11      3042
   macro avg       0.48      0.49      0.10      3042
weighted avg       0.81      0.11      0.07      3042



In [19]:
##train a CNN with activation function=relu, optimizer=adam, loss=mean squared error, batch size=100 and 10 epochs.

import tensorflow as tf
from tensorflow import keras

new_model1 = keras.models.Sequential()

new_model1.add(keras.layers.Conv3D(32, (3,3,1)  , activation = 'relu' , padding = 'same' , input_shape=(8,8,8,1)))
new_model1.add(keras.layers.Conv3D(32, (3,3,1) , padding = 'same' , activation = 'relu' ))
new_model1.add(keras.layers.MaxPooling3D((2,2,1) , padding = 'same'))
new_model1.add(keras.layers.Dropout(0.25))
new_model1.add(keras.layers.Conv3D(64, (3,3,1) , padding = 'same' , activation = 'relu' ))
new_model1.add(keras.layers.Conv3D(64, (3,3,1) , padding = 'same' , activation = 'relu' ))
new_model1.add(keras.layers.MaxPooling3D((2,2,1) , padding = 'same'))
new_model1.add(keras.layers.Dropout(0.25))
new_model1.add(keras.layers.Flatten())

new_model1.add(keras.layers.Dense(120 , activation = 'relu'))
new_model1.add(keras.layers.Dense(60 , activation = 'relu'))
new_model1.add(keras.layers.Dense(2, activation = 'softmax'))
              
new_model1.compile(optimizer = 'adam' , loss = 'mean_squared_error' , metrics = ['accuracy' ])

new_model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 8, 8, 8, 32)       320       
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 8, 8, 8, 32)       9248      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 4, 8, 32)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 8, 32)       0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 4, 4, 8, 64)       18496     
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 4, 4, 8, 64)       36928     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 2, 2, 8, 64)       0         
__________

In [20]:
X_train = X_train.reshape(-1,8,8,8,1)
X_test = X_test.reshape(-1,8,8,8,1)

In [21]:
history = new_model1.fit(X_train , Y_train, epochs = 10 , batch_size = 100, validation_data = (X_test,Y_test))

Train on 9125 samples, validate on 3042 samples
Epoch 1/10
9125/9125 [==============================] - 53s 6ms/sample - loss: 0.4977 - acc: 0.7316 - val_loss: 0.5000 - val_acc: 1.0000
Epoch 2/10
9125/9125 [==============================] - 52s 6ms/sample - loss: 0.5000 - acc: 1.0000 - val_loss: 0.5000 - val_acc: 1.0000
Epoch 3/10
9125/9125 [==============================] - 53s 6ms/sample - loss: 0.5000 - acc: 1.0000 - val_loss: 0.5000 - val_acc: 1.0000
Epoch 4/10
9125/9125 [==============================] - 50s 5ms/sample - loss: 0.5000 - acc: 1.0000 - val_loss: 0.5000 - val_acc: 1.0000
Epoch 5/10
9125/9125 [==============================] - 51s 6ms/sample - loss: 0.5000 - acc: 1.0000 - val_loss: 0.5000 - val_acc: 1.0000
Epoch 6/10
9125/9125 [==============================] - 47s 5ms/sample - loss: 0.5000 - acc: 1.0000 - val_loss: 0.5000 - val_acc: 1.0000
Epoch 7/10
9125/9125 [==============================] - 50s 5ms/sample - loss: 0.5000 - acc: 1.0000 - val_loss: 0.5000 - val_acc: 

In [22]:
from sklearn.metrics import confusion_matrix

Y_predicted = np.argmax( new_model1.predict(X_test ), axis=1 )
cm = confusion_matrix(Y_test, Y_predicted)
print(cm)

[[2760    0]
 [ 282    0]]


In [23]:
from sklearn.metrics import classification_report
Y_temp=Y[:len(Y_predicted)]
clf_report = classification_report(Y_temp, Y_predicted)
print(clf_report)

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      2794
         1.0       0.00      0.00      0.00       248

    accuracy                           0.92      3042
   macro avg       0.46      0.50      0.48      3042
weighted avg       0.84      0.92      0.88      3042



C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
